<a href="https://colab.research.google.com/github/jorgewellison/classifiers-tcc/blob/main/Pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importação das bibliotecas**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix

In [94]:
pd.set_option('display.max_rows', 300)

**Tratamento dos indicadores do IDHM**

In [3]:
atlas = pd.read_excel("/content/drive/My Drive/Colab Notebooks/covid-idhm/atlas2013_dadosbrutos_pt.xlsx", sheet_name=1, 
          usecols=['ANO', 'UF', 'Codmun7', 'Município', 'ESPVIDA', 'FECTOT', 'MORT1', 'SOBRE60', 'T_ENV', 'E_ANOSESTUDO', 'T_ANALF11A14', 'T_ANALF25M', 'T_ATRASO_0_BASICO', 'T_ATRASO_2_BASICO', 'T_FLPRE', 'T_FLSUPER', 'T_FREQSUPER1517', 'T_SUPER25M', 'T_MED25M', 'GINI', 'PIND', 'PINDCRI', 'PMPOB', 'PMPOBCRI', 'RDPC', 'CPR', 'EMP', 'P_AGRO', 'P_COM', 'P_CONSTR', 'P_EXTR', 'P_FORMAL', 'P_FUND', 'P_MED', 'P_SERV', 'P_SIUP', 'P_SUPER', 'T_DES2529', 'TRABCC', 'TRABPUB', 'TRABSC', 'T_AGUA', 'T_BANAGUA', 'T_DENS', 'T_LIXO', 'T_LUZ', 'AGUA_ESGOTO', 'PAREDE', 'T_CRIFUNDIN_TODOS', 'T_FORA4A5', 'T_FORA6A14', 'T_FUNDIN_TODOS', 'T_FUNDIN_TODOS_MMEIO', 'T_FUNDIN18MINF', 'T_M10A14CF', 'T_M15A17CF', 'T_MULCHEFEFIF014', 'T_NESTUDA_NTRAB_MMEIO', 'T_OCUPDESLOC_1', 'T_RMAXIDOSO', 'T_SLUZ', 'HOMEMTOT', 'MULHERTOT', 'pesoRUR', 'pesotot', 'pesourb', 'POP', 'IDHM', 'IDHM_E', 'IDHM_L', 'IDHM_R'], 
          dtype={'UF': str}).rename(columns={'ESPVIDA': 'expec_vida', 'FECTOT': 'taxa_fecund', 'SOBRE60': '%sobreviver_60a', 'T_ENV': 'taxa_envelh', 'E_ANOSESTUDO': 'expec_anos_estudo_18'})
atlasbr = atlas.loc[(atlas['ANO'] == 2010)]

In [ ]:
atlasbr.head()

,ANO,UF,Codmun7,Município,expec_vida,taxa_fecund,MORT1,%sobreviver_60a,taxa_envelh,expec_anos_estudo_18,T_ANALF11A14,T_ANALF25M,T_ATRASO_0_BASICO,T_ATRASO_2_BASICO,T_FLPRE,T_FLSUPER,T_FREQSUPER1517,T_MED25M,T_SUPER25M,GINI,PIND,PINDCRI,PMPOB,PMPOBCRI,RDPC,CPR,EMP,P_AGRO,P_COM,P_CONSTR,P_EXTR,P_FORMAL,P_FUND,P_MED,P_SERV,P_SIUP,P_SUPER,T_DES2529,TRABCC,TRABPUB,TRABSC,T_AGUA,T_BANAGUA,T_DENS,T_LIXO,T_LUZ,AGUA_ESGOTO,PAREDE,T_CRIFUNDIN_TODOS,T_FORA4A5,T_FORA6A14,T_FUNDIN_TODOS,T_FUNDIN_TODOS_MMEIO,T_FUNDIN18MINF,T_M10A14CF,T_M15A17CF,T_MULCHEFEFIF014,T_NESTUDA_NTRAB_MMEIO,T_OCUPDESLOC_1,T_RMAXIDOSO,T_SLUZ,HOMEMTOT,MULHERTOT,pesoRUR,pesotot,pesourb,POP,IDHM,IDHM_E,IDHM_L,IDHM_R
11130,2010,11,1100015,ALTA FLORESTA D'OESTE,70.75,2.24,23.8,83.18,5.84,8.67,1.22,15.76,57.62,22.03,34.15,10.50,2.73,19.19,5.51,0.58,14.29,19.01,26.04,37.33,476.99,34.73,2.83,38.56,13.78,5.31,0.00,41.68,45.49,31.05,33.18,0.69,7.72,5.06,25.17,5.31,23.17,93.69,80.20,22.58,94.05,93.98,9.31,7.18,45.73,60.82,4.32,40.27,24.81,54.06,0.00,6.75,14.51,16.12,1.15,2.20,6.02,12656,11736,10422,24392,13970,24097,0.641,0.526,0.763,0.657
11131,2010,11,1100023,ARIQUEMES,73.36,1.91,19.2,87.69,4.36,9.18,1.55,10.48,58.43,19.35,40.96,12.91,0.18,31.45,8.13,0.53,4.36,6.65,11.54,18.76,689.95,27.66,2.30,14.97,17.99,8.68,3.32,52.81,57.26,39.76,40.06,0.77,9.29,5.85,41.15,6.30,19.21,98.54,85.35,27.15,96.72,98.58,11.40,5.62,29.58,44.66,2.79,26.59,12.27,37.81,0.00,9.10,18.07,9.67,0.16,1.40,1.42,45543,44810,13828,90353,76525,89766,0.702,0.600,0.806,0.716
11132,2010,11,1100031,CABIXI,70.39,2.15,24.8,82.57,6.87,10.09,0.62,16.78,66.22,15.59,58.34,9.86,1.05,20.42,4.93,0.51,7.27,8.51,21.20,31.01,457.17,32.98,0.67,45.49,6.46,3.62,0.00,39.38,44.10,30.24,36.91,0.52,6.25,3.60,24.03,7.79,22.12,95.49,91.63,19.93,99.14,96.36,3.31,0.84,41.25,33.08,1.97,38.52,21.53,54.79,1.12,5.47,7.03,16.59,0.65,3.10,3.64,3266,3047,3620,6313,2693,6303,0.650,0.559,0.757,0.650
11133,2010,11,1100049,CACOAL,74.27,1.90,14.3,88.56,5.57,9.74,1.01,10.95,66.69,15.57,51.33,17.45,0.44,28.06,8.71,0.57,5.97,8.77,13.08,20.03,738.06,24.97,2.09,19.69,19.00,7.16,0.25,55.07,55.78,37.67,39.07,0.85,10.29,6.22,40.10,7.70,18.21,97.96,93.44,20.50,98.17,98.89,1.85,3.39,25.72,37.59,2.39,26.16,11.58,39.87,0.68,6.03,12.80,9.64,0.43,1.84,1.11,39124,39450,16653,78574,61921,78141,0.718,0.620,0.821,0.727
11134,2010,11,1100056,CEREJEIRAS,72.94,2.12,18.1,86.72,6.44,10.22,1.14,13.51,72.15,12.93,49.74,10.77,2.02,24.06,7.09,0.50,4.72,6.93,13.70,22.08,577.18,28.46,2.36,26.67,14.08,8.60,0.06,45.54,49.50,34.10,38.39,0.57,8.53,7.22,24.74,9.77,25.05,97.53,86.87,16.66,91.89,98.85,5.34,0.85,35.81,38.73,1.46,32.21,15.53,46.96,0.00,1.77,25.88,10.16,0.38,1.43,1.15,8551,8478,2610,17029,14419,16942,0.692,0.602,0.799,0.688


**Carregando dados de mortalidade e de mobilidade**

In [115]:
base_mort = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid-idhm/cases-brazil-cities.csv", usecols=['ibgeID', 'deaths_per_100k_inhabitants'])
base_mort.head()

,ibgeID,deaths_per_100k_inhabitants
0,5200050,79.79027
1,3100104,14.30820
2,5200100,54.88474
3,1500107,71.02183
4,3100203,8.60696


In [28]:
mob = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid-idhm/Social Distancing Index by Cities (1).csv", parse_dates = ['dt'])
mobbr = mob[['city_name', 'isolated', 'dt']]

In [29]:
mobbr

,city_name,isolated,dt
0,Capivari do Sul,0.176474,2020-11-04
1,Iperó,0.346263,2020-11-04
2,Nova Europa,0.258624,2020-11-04
3,Campina Grande do Sul,0.305752,2020-11-04
4,Camocim,0.392672,2020-11-04
...,...,...,...
1127908,Umarizal,0.321429,2020-02-01
1127909,Teolândia,0.238994,2020-02-01
1127910,Sebastião Laranjeiras,0.342466,2020-02-01
1127911,Roncador,0.339623,2020-02-01


In [31]:
mobbr.head()

,city_name,isolated,dt
0,Capivari do Sul,0.176474,2020-11-04
1,Iperó,0.346263,2020-11-04
2,Nova Europa,0.258624,2020-11-04
3,Campina Grande do Sul,0.305752,2020-11-04
4,Camocim,0.392672,2020-11-04


In [32]:
start_date  =  '2020-03-01'
mask = mobbr['dt'] >= start_date
mobbr_marco_novembro = mobbr.loc[mask]
mobbr_marco_novembro

,city_name,isolated,dt
0,Capivari do Sul,0.176474,2020-11-04
1,Iperó,0.346263,2020-11-04
2,Nova Europa,0.258624,2020-11-04
3,Campina Grande do Sul,0.305752,2020-11-04
4,Camocim,0.392672,2020-11-04
...,...,...,...
1012573,Santo Antônio do Descoberto,0.402174,2020-03-01
1012574,Santa Luzia d'Oeste,0.320000,2020-03-01
1012575,Águas Mornas,0.405660,2020-03-01
1012576,Carutapera,0.360656,2020-03-01


**Média durante toda a pandemia**

In [37]:
media_municipio = mobbr_marco_novembro.groupby('city_name')['isolated'].mean().reset_index(name='media_mes')
media_municipio

,city_name,media_mes
0,Abadia de Goiás,0.362751
1,Abadia dos Dourados,0.420707
2,Abadiânia,0.388448
3,Abaetetuba,0.417864
4,Abaeté,0.352498
...,...,...
4554,Álvares Machado,0.385168
4555,Álvaro de Carvalho,0.383411
4556,Ângulo,0.361838
4557,Érico Cardoso,0.416686


**Adicionando dados de mobilidade e mortalidade aos dados da COVID-19**

In [107]:
base = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid-idhm/covid19-setembro.csv") 

In [ ]:
base_mort = pd.merge(base_mort, base, left_on="ibgeID", right_on="city_ibge_code", how="left")
base_mort.isnull().sum()

In [129]:
base_mob = pd.merge(media_municipio, base_mort, left_on="city_name", right_on="city", how='left')
base_mob.head()

,city_name,media_mes,ibgeID,deaths_per_100k_inhabitants,epidemiological_week_x,date_x,order_for_place_x,state_x,city_x,city_ibge_code_x,place_type_x,last_available_confirmed_x,last_available_confirmed_per_100k_inhabitants_x,new_confirmed_x,last_available_deaths_x,new_deaths_x,last_available_death_rate_x,estimated_population_2019_x,is_last_x,is_repeated_x,epidemiological_week_y,date_y,order_for_place_y,state_y,city_y,city_ibge_code_y,place_type_y,last_available_confirmed_y,last_available_confirmed_per_100k_inhabitants_y,new_confirmed_y,last_available_deaths_y,new_deaths_y,last_available_death_rate_y,estimated_population_2019_y,is_last_y,is_repeated_y,epidemiological_week,date,order_for_place,state,city,city_ibge_code,place_type,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,new_confirmed,last_available_deaths,new_deaths,last_available_death_rate,estimated_population_2019,is_last,is_repeated
0,Abadia de Goiás,0.362751,5200050.0,79.79027,40.0,2020-09-27,130.0,GO,Abadia de Goiás,5200050.0,city,534.0,6086.85740,0.0,7.0,0.0,0.0131,8773.0,True,False,40.0,2020-09-27,130.0,GO,Abadia de Goiás,5200050.0,city,534.0,6086.85740,0.0,7.0,0.0,0.0131,8773.0,True,False,40.0,2020-09-27,130.0,GO,Abadia de Goiás,5200050.0,city,534.0,6086.85740,0.0,7.0,0.0,0.0131,8773.0,True,False
1,Abadia dos Dourados,0.420707,3100104.0,14.30820,40.0,2020-09-27,143.0,MG,Abadia dos Dourados,3100104.0,city,86.0,1230.50508,0.0,1.0,0.0,0.0116,6989.0,True,False,40.0,2020-09-27,143.0,MG,Abadia dos Dourados,3100104.0,city,86.0,1230.50508,0.0,1.0,0.0,0.0116,6989.0,True,False,40.0,2020-09-27,143.0,MG,Abadia dos Dourados,3100104.0,city,86.0,1230.50508,0.0,1.0,0.0,0.0116,6989.0,True,False
2,Abadiânia,0.388448,5200100.0,54.88474,40.0,2020-09-27,122.0,GO,Abadiânia,5200100.0,city,127.0,633.66929,0.0,10.0,0.0,0.0787,20042.0,True,False,40.0,2020-09-27,122.0,GO,Abadiânia,5200100.0,city,127.0,633.66929,0.0,10.0,0.0,0.0787,20042.0,True,False,40.0,2020-09-27,122.0,GO,Abadiânia,5200100.0,city,127.0,633.66929,0.0,10.0,0.0,0.0787,20042.0,True,False
3,Abaetetuba,0.417864,1500107.0,71.02183,40.0,2020-09-27,181.0,PA,Abaetetuba,1500107.0,city,3183.0,2018.41495,0.0,112.0,0.0,0.0352,157698.0,False,False,40.0,2020-09-27,181.0,PA,Abaetetuba,1500107.0,city,3183.0,2018.41495,0.0,112.0,0.0,0.0352,157698.0,False,False,40.0,2020-09-27,181.0,PA,Abaetetuba,1500107.0,city,3183.0,2018.41495,0.0,112.0,0.0,0.0352,157698.0,False,False
4,Abaeté,0.352498,3100203.0,8.60696,40.0,2020-09-27,169.0,MG,Abaeté,3100203.0,city,52.0,223.78104,0.0,1.0,0.0,0.0192,23237.0,True,False,40.0,2020-09-27,169.0,MG,Abaeté,3100203.0,city,52.0,223.78104,0.0,1.0,0.0,0.0192,23237.0,True,False,40.0,2020-09-27,169.0,MG,Abaeté,3100203.0,city,52.0,223.78104,0.0,1.0,0.0,0.0192,23237.0,True,False


In [130]:
base_mob.isnull().sum()

city_name                                           0
media_mes                                           0
ibgeID                                             62
deaths_per_100k_inhabitants                        62
epidemiological_week_x                             62
date_x                                             62
order_for_place_x                                  62
state_x                                            62
city_x                                             62
city_ibge_code_x                                   62
place_type_x                                       62
last_available_confirmed_x                         62
last_available_confirmed_per_100k_inhabitants_x    63
new_confirmed_x                                    62
last_available_deaths_x                            62
new_deaths_x                                       62
last_available_death_rate_x                        62
estimated_population_2019_x                        62
is_last_x                   

In [131]:
pd.isnull(base_mob['last_available_confirmed_per_100k_inhabitants'])
base_mob.loc[pd.isnull(base_mob['last_available_confirmed_per_100k_inhabitants'])]

,city_name,media_mes,ibgeID,deaths_per_100k_inhabitants,epidemiological_week_x,date_x,order_for_place_x,state_x,city_x,city_ibge_code_x,place_type_x,last_available_confirmed_x,last_available_confirmed_per_100k_inhabitants_x,new_confirmed_x,last_available_deaths_x,new_deaths_x,last_available_death_rate_x,estimated_population_2019_x,is_last_x,is_repeated_x,epidemiological_week_y,date_y,order_for_place_y,state_y,city_y,city_ibge_code_y,place_type_y,last_available_confirmed_y,last_available_confirmed_per_100k_inhabitants_y,new_confirmed_y,last_available_deaths_y,new_deaths_y,last_available_death_rate_y,estimated_population_2019_y,is_last_y,is_repeated_y,epidemiological_week,date,order_for_place,state,city,city_ibge_code,place_type,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,new_confirmed,last_available_deaths,new_deaths,last_available_death_rate,estimated_population_2019,is_last,is_repeated
88,Alta Floresta d'Oeste,0.432107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,Alvorada d'Oeste,0.432763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,Alvorada do Gurgueia,0.389974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,Araçás,0.404736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,Arez,0.419025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,Assu,0.377513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343,Atílio Vivácqua,0.378292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,Barra d'Alcântara,0.368507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,Biritiba Mirim,0.404515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528,Boa Saúde,0.408070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
base_mob = base_mob.drop([88,130,131,288,303,334,343,412,518,528,544,545,603,612,1105,1124,1142,1338,1380,1415,1446,1458,1564,1605,1688,1742,1943,1989,2043,2327,2380,2386,2394,2422,2445,2475,2495,2630,2799,2875,3146,3162,3170,3370,3561,3710,3821,3822,3897,3926,4111,4119,4126,4182,4269,4274,4332,4389,4390,4396,4426,4473,4715])

**Remodelando o dataframe dos casos de covid**

In [134]:
cols = list(base_mob.columns.values)
df = base_mob[['media_mes', 'city_ibge_code', 'estimated_population_2019', 'last_available_confirmed_per_100k_inhabitants', 'deaths_per_100k_inhabitants']]
df.fillna(0,inplace=True)
df.isnull().sum()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


media_mes                                        0
city_ibge_code                                   0
estimated_population_2019                        0
last_available_confirmed_per_100k_inhabitants    0
deaths_per_100k_inhabitants                      0
dtype: int64

**Junção da tabela de métricas de indicadores do IDHM (Atlas) + dataFrame covid**

In [140]:
idhmcovid = pd.merge(atlasbr, df, left_on="Codmun7", right_on="city_ibge_code", how="left")
idhmcovid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 0 to 5564
Data columns (total 76 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   ANO                                            5565 non-null   int64  
 1   UF                                             5565 non-null   object 
 2   Codmun7                                        5565 non-null   int64  
 3   Município                                      5565 non-null   object 
 4   expec_vida                                     5565 non-null   float64
 5   taxa_fecund                                    5565 non-null   float64
 6   MORT1                                          5565 non-null   float64
 7   %sobreviver_60a                                5565 non-null   float64
 8   taxa_envelh                                    5565 non-null   float64
 9   expec_anos_estudo_18                           5565 

In [141]:
idhmcovid.loc[pd.isnull(idhmcovid['last_available_confirmed_per_100k_inhabitants'])]

,ANO,UF,Codmun7,Município,expec_vida,taxa_fecund,MORT1,%sobreviver_60a,taxa_envelh,expec_anos_estudo_18,T_ANALF11A14,T_ANALF25M,T_ATRASO_0_BASICO,T_ATRASO_2_BASICO,T_FLPRE,T_FLSUPER,T_FREQSUPER1517,T_MED25M,T_SUPER25M,GINI,PIND,PINDCRI,PMPOB,PMPOBCRI,RDPC,CPR,EMP,P_AGRO,P_COM,P_CONSTR,P_EXTR,P_FORMAL,P_FUND,P_MED,P_SERV,P_SIUP,P_SUPER,T_DES2529,TRABCC,TRABPUB,TRABSC,T_AGUA,T_BANAGUA,T_DENS,T_LIXO,T_LUZ,AGUA_ESGOTO,PAREDE,T_CRIFUNDIN_TODOS,T_FORA4A5,T_FORA6A14,T_FUNDIN_TODOS,T_FUNDIN_TODOS_MMEIO,T_FUNDIN18MINF,T_M10A14CF,T_M15A17CF,T_MULCHEFEFIF014,T_NESTUDA_NTRAB_MMEIO,T_OCUPDESLOC_1,T_RMAXIDOSO,T_SLUZ,HOMEMTOT,MULHERTOT,pesoRUR,pesotot,pesourb,POP,IDHM,IDHM_E,IDHM_L,IDHM_R,media_mes,city_ibge_code,estimated_population_2019,last_available_confirmed_per_100k_inhabitants,deaths_per_100k_inhabitants
0,2010,11,1100015,ALTA FLORESTA D'OESTE,70.75,2.24,23.8,83.18,5.84,8.67,1.22,15.76,57.62,22.03,34.15,10.50,2.73,19.19,5.51,0.58,14.29,19.01,26.04,37.33,476.99,34.73,2.83,38.56,13.78,5.31,0.00,41.68,45.49,31.05,33.18,0.69,7.72,5.06,25.17,5.31,23.17,93.69,80.20,22.58,94.05,93.98,9.31,7.18,45.73,60.82,4.32,40.27,24.81,54.06,0.00,6.75,14.51,16.12,1.15,2.20,6.02,12656,11736,10422,24392,13970,24097,0.641,0.526,0.763,0.657,NaN,NaN,NaN,NaN,NaN
2,2010,11,1100031,CABIXI,70.39,2.15,24.8,82.57,6.87,10.09,0.62,16.78,66.22,15.59,58.34,9.86,1.05,20.42,4.93,0.51,7.27,8.51,21.20,31.01,457.17,32.98,0.67,45.49,6.46,3.62,0.00,39.38,44.10,30.24,36.91,0.52,6.25,3.60,24.03,7.79,22.12,95.49,91.63,19.93,99.14,96.36,3.31,0.84,41.25,33.08,1.97,38.52,21.53,54.79,1.12,5.47,7.03,16.59,0.65,3.10,3.64,3266,3047,3620,6313,2693,6303,0.650,0.559,0.757,0.650,NaN,NaN,NaN,NaN,NaN
8,2010,11,1100098,ESPIGÃO D'OESTE,74.15,2.22,15.4,88.54,5.21,9.55,2.17,14.87,65.19,16.09,30.29,9.83,0.49,18.24,5.72,0.52,6.60,9.79,13.86,21.58,590.80,28.04,1.09,29.68,16.86,4.42,0.32,41.78,44.34,25.08,31.45,0.63,6.75,6.12,29.95,6.66,24.30,93.70,84.71,19.97,91.07,96.70,7.66,5.89,39.66,54.18,4.09,35.20,17.10,48.62,1.59,7.73,24.70,10.40,0.53,1.88,3.30,14606,14123,8119,28729,20610,28510,0.672,0.536,0.819,0.691,NaN,NaN,NaN,NaN,NaN
12,2010,11,1100130,MACHADINHO D'OESTE,70.32,2.54,25.0,82.45,4.05,7.80,1.68,15.34,46.45,28.82,30.11,4.75,1.74,13.87,3.72,0.55,17.59,22.26,28.55,38.74,401.42,35.99,0.82,48.35,9.99,3.37,0.19,31.93,36.72,20.90,27.65,0.41,4.84,1.86,19.81,5.02,21.11,92.56,69.34,28.87,91.21,88.10,17.19,4.07,54.59,61.46,6.78,48.97,31.81,58.76,0.60,10.67,27.21,17.55,0.61,3.44,11.90,16536,14599,14962,31135,16173,31020,0.596,0.446,0.755,0.629,NaN,NaN,NaN,NaN,NaN
13,2010,11,1100148,NOVA BRASILÂNDIA D'OESTE,70.75,2.03,23.8,83.18,5.49,9.44,0.78,14.86,57.52,20.71,52.82,5.83,1.02,18.76,4.76,0.56,10.30,12.69,23.16,31.29,499.59,46.87,2.41,52.88,11.50,4.46,0.07,29.12,42.92,26.90,21.60,0.22,6.02,4.76,17.95,5.48,14.91,94.07,74.60,20.31,94.84,94.95,9.60,0.88,39.16,43.18,2.52,37.74,21.31,57.97,0.00,5.69,10.99,12.46,0.65,2.02,5.05,10344,9530,11494,19874,8380,19526,0.643,0.524,0.763,0.664,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5545,2010,52,5220702,SÍTIO D'ABADIA,73.09,2.84,15.9,80.50,5.91,9.75,4.51,31.49,59.90,23.56,39.72,0.00,0.00,15.09,2.82,0.56,19.10,23.64,36.43,52.59,314.05,23.27,0.00,48.08,5.16,3.35,0.00,28.92,36.45,21.71,31.89,0.00,4.42,2.02,11.23,16.53,25.14,78.73,84.44,20.82,95.54,95.02,4.95,3.39,42.02,18.15,2.45,40.19,29.32,58.87,0.00,3.75,27.68,19.57,0.73,2.92,4.98,1519,1306,1831,2825,994,2767,0.617,0.497,0.802,0.590,NaN,NaN,NaN,NaN,NaN
5547,2010,52,5221080,TERESINA DE GOIÁS,74.00,2.86,14.6,81.88,7.00,9.78,2.86,21.84,61.71,22.84,59.00,3.91,0.00,25.43,5.08,0.52,9.59,16.20,29.11,46.47,394.33,18.48,0.89,32.65,11.90,5.29,0.97,36.02,50.33,35.23,43.73,0.36,6.40,3.91,18.75,13.93,32.48,80.42,84.38,36.69,99.09,94.76,4.89,6.17,39.36

**Calculando média de coluna que serão utilizadas**

In [142]:
casosMedios = idhmcovid['last_available_confirmed_per_100k_inhabitants'].mean()
casosMedios = math.floor(casosMedios)

populacaoMedia = idhmcovid['estimated_population_2019'].mean()
populacaoMedia = math.floor(populacaoMedia)

media_mes = idhmcovid['media_mes'].mean()
media_mes = math.floor(media_mes)

mediaMortes = idhmcovid['deaths_per_100k_inhabitants'].mean()
mediaMortes = math.floor(mediaMortes)



**Trocando valores nulos pela média**

In [143]:
idhmcovid.update(idhmcovid['last_available_confirmed_per_100k_inhabitants'].fillna(casosMedios))
idhmcovid.update(idhmcovid['estimated_population_2019'].fillna(populacaoMedia))
idhmcovid.update(idhmcovid['media_mes'].fillna(media_mes))
idhmcovid.update(idhmcovid['deaths_per_100k_inhabitants'].fillna(mediaMortes))
idhmcovid.update(idhmcovid['city_ibge_code'].fillna(0))


In [144]:
idhmcovid.isnull().sum()

ANO                                              0
UF                                               0
Codmun7                                          0
Município                                        0
expec_vida                                       0
taxa_fecund                                      0
MORT1                                            0
%sobreviver_60a                                  0
taxa_envelh                                      0
expec_anos_estudo_18                             0
T_ANALF11A14                                     0
T_ANALF25M                                       0
T_ATRASO_0_BASICO                                0
T_ATRASO_2_BASICO                                0
T_FLPRE                                          0
T_FLSUPER                                        0
T_FREQSUPER1517                                  0
T_MED25M                                         0
T_SUPER25M                                       0
GINI                           

**Separando apenas as colunas necessárias para selecionar os previsores e a classe**

In [149]:
cols = list(idhmcovid.columns.values)
idhmcovid_classif = idhmcovid[['deaths_per_100k_inhabitants', 'media_mes', 'expec_vida', 'taxa_fecund','MORT1','%sobreviver_60a','taxa_envelh','expec_anos_estudo_18','T_ANALF11A14','T_ANALF25M','T_ATRASO_0_BASICO','T_ATRASO_2_BASICO','T_FLPRE','T_FLSUPER','T_FREQSUPER1517','T_MED25M','T_SUPER25M','GINI','PIND','PINDCRI','PMPOB','PMPOBCRI','RDPC','CPR','EMP','P_AGRO','P_COM','P_CONSTR','P_EXTR','P_FORMAL','P_FUND','P_MED','P_SERV','P_SIUP','P_SUPER','T_DES2529','TRABCC','TRABPUB','TRABSC','T_AGUA','T_BANAGUA','T_DENS','T_LIXO','T_LUZ','AGUA_ESGOTO','PAREDE','T_CRIFUNDIN_TODOS','T_FORA4A5','T_FORA6A14','T_FUNDIN_TODOS','T_FUNDIN_TODOS_MMEIO','T_FUNDIN18MINF','T_M10A14CF','T_M15A17CF','T_MULCHEFEFIF014','T_NESTUDA_NTRAB_MMEIO','T_OCUPDESLOC_1','T_RMAXIDOSO','T_SLUZ','HOMEMTOT','MULHERTOT','pesoRUR','pesotot','pesourb','POP','IDHM','IDHM_E','IDHM_L','IDHM_R','estimated_population_2019', 'last_available_confirmed_per_100k_inhabitants']]
idhmcovid_classif.head()

,deaths_per_100k_inhabitants,media_mes,expec_vida,taxa_fecund,MORT1,%sobreviver_60a,taxa_envelh,expec_anos_estudo_18,T_ANALF11A14,T_ANALF25M,T_ATRASO_0_BASICO,T_ATRASO_2_BASICO,T_FLPRE,T_FLSUPER,T_FREQSUPER1517,T_MED25M,T_SUPER25M,GINI,PIND,PINDCRI,PMPOB,PMPOBCRI,RDPC,CPR,EMP,P_AGRO,P_COM,P_CONSTR,P_EXTR,P_FORMAL,P_FUND,P_MED,P_SERV,P_SIUP,P_SUPER,T_DES2529,TRABCC,TRABPUB,TRABSC,T_AGUA,T_BANAGUA,T_DENS,T_LIXO,T_LUZ,AGUA_ESGOTO,PAREDE,T_CRIFUNDIN_TODOS,T_FORA4A5,T_FORA6A14,T_FUNDIN_TODOS,T_FUNDIN_TODOS_MMEIO,T_FUNDIN18MINF,T_M10A14CF,T_M15A17CF,T_MULCHEFEFIF014,T_NESTUDA_NTRAB_MMEIO,T_OCUPDESLOC_1,T_RMAXIDOSO,T_SLUZ,HOMEMTOT,MULHERTOT,pesoRUR,pesotot,pesourb,POP,IDHM,IDHM_E,IDHM_L,IDHM_R,estimated_population_2019,last_available_confirmed_per_100k_inhabitants
0,39.00000,0.000000,70.75,2.24,23.8,83.18,5.84,8.67,1.22,15.76,57.62,22.03,34.15,10.50,2.73,19.19,5.51,0.58,14.29,19.01,26.04,37.33,476.99,34.73,2.83,38.56,13.78,5.31,0.00,41.68,45.49,31.05,33.18,0.69,7.72,5.06,25.17,5.31,23.17,93.69,80.20,22.58,94.05,93.98,9.31,7.18,45.73,60.82,4.32,40.27,24.81,54.06,0.00,6.75,14.51,16.12,1.15,2.20,6.02,12656,11736,10422,24392,13970,24097,0.641,0.526,0.763,0.657,43207.0,1813.00000
1,86.22048,0.379193,73.36,1.91,19.2,87.69,4.36,9.18,1.55,10.48,58.43,19.35,40.96,12.91,0.18,31.45,8.13,0.53,4.36,6.65,11.54,18.76,689.95,27.66,2.30,14.97,17.99,8.68,3.32,52.81,57.26,39.76,40.06,0.77,9.29,5.85,41.15,6.30,19.21,98.54,85.35,27.15,96.72,98.58,11.40,5.62,29.58,44.66,2.79,26.59,12.27,37.81,0.00,9.10,18.07,9.67,0.16,1.40,1.42,45543,44810,13828,90353,76525,89766,0.702,0.600,0.806,0.716,107863.0,4904.36943
2,39.00000,0.000000,70.39,2.15,24.8,82.57,6.87,10.09,0.62,16.78,66.22,15.59,58.34,9.86,1.05,20.42,4.93,0.51,7.27,8.51,21.20,31.01,457.17,32.98,0.67,45.49,6.46,3.62,0.00,39.38,44.10,30.24,36.91,0.52,6.25,3.60,24.03,7.79,22.12,95.49,91.63,19.93,99.14,96.36,3.31,0.84,41.25,33.08,1.97,38.52,21.53,54.79,1.12,5.47,7.03,16.59,0.65,3.10,3.64,3266,3047,3620,6313,2693,6303,0.650,0.559,0.757,0.650,43207.0,1813.00000
3,39.83177,0.401504,74.27,1.90,14.3,88.56,5.57,9.74,1.01,10.95,66.69,15.57,51.33,17.45,0.44,28.06,8.71,0.57,5.97,8.77,13.08,20.03,738.06,24.97,2.09,19.69,19.00,7.16,0.25,55.07,55.78,37.67,39.07,0.85,10.29,6.22,40.10,7.70,18.21,97.96,93.44,20.50,98.17,98.89,1.85,3.39,25.72,37.59,2.39,26.16,11.58,39.87,0.68,6.03,12.80,9.64,0.43,1.84,1.11,39124,39450,16653,78574,61921,78141,0.718,0.620,0.821,0.727,85359.0,2653.49875
4,36.75795,0.384490,72.94,2.12,18.1,86.72,6.44,10.22,1.14,13.51,72.15,12.93,49.74,10.77,2.02,24.06,7.09,0.50,4.72,6.93,13.70,22.08,577.18,28.46,2.36,26.67,14.08,8.60,0.06,45.54,49.50,34.10,38.39,0.57,8.53,7.22,24.74,9.77,25.05,97.53,86.87,16.66,91.89,98.85,5.34,0.85,35.81,38.73,1.46,32.21,15.53,46.96,0.00,1.77,25.88,10.16,0.38,1.43,1.15,8551,8478,2610,17029,14419,16942,0.692,0.602,0.799,0.688,16323.0,1519.32855


In [151]:
idhmcovid_classif.head()

,deaths_per_100k_inhabitants,media_mes,expec_vida,taxa_fecund,MORT1,%sobreviver_60a,taxa_envelh,expec_anos_estudo_18,T_ANALF11A14,T_ANALF25M,T_ATRASO_0_BASICO,T_ATRASO_2_BASICO,T_FLPRE,T_FLSUPER,T_FREQSUPER1517,T_MED25M,T_SUPER25M,GINI,PIND,PINDCRI,PMPOB,PMPOBCRI,RDPC,CPR,EMP,P_AGRO,P_COM,P_CONSTR,P_EXTR,P_FORMAL,P_FUND,P_MED,P_SERV,P_SIUP,P_SUPER,T_DES2529,TRABCC,TRABPUB,TRABSC,T_AGUA,T_BANAGUA,T_DENS,T_LIXO,T_LUZ,AGUA_ESGOTO,PAREDE,T_CRIFUNDIN_TODOS,T_FORA4A5,T_FORA6A14,T_FUNDIN_TODOS,T_FUNDIN_TODOS_MMEIO,T_FUNDIN18MINF,T_M10A14CF,T_M15A17CF,T_MULCHEFEFIF014,T_NESTUDA_NTRAB_MMEIO,T_OCUPDESLOC_1,T_RMAXIDOSO,T_SLUZ,HOMEMTOT,MULHERTOT,pesoRUR,pesotot,pesourb,POP,IDHM,IDHM_E,IDHM_L,IDHM_R,estimated_population_2019,last_available_confirmed_per_100k_inhabitants
0,39.00000,0.000000,70.75,2.24,23.8,83.18,5.84,8.67,1.22,15.76,57.62,22.03,34.15,10.50,2.73,19.19,5.51,0.58,14.29,19.01,26.04,37.33,476.99,34.73,2.83,38.56,13.78,5.31,0.00,41.68,45.49,31.05,33.18,0.69,7.72,5.06,25.17,5.31,23.17,93.69,80.20,22.58,94.05,93.98,9.31,7.18,45.73,60.82,4.32,40.27,24.81,54.06,0.00,6.75,14.51,16.12,1.15,2.20,6.02,12656,11736,10422,24392,13970,24097,0.641,0.526,0.763,0.657,43207.0,1813.00000
1,86.22048,0.379193,73.36,1.91,19.2,87.69,4.36,9.18,1.55,10.48,58.43,19.35,40.96,12.91,0.18,31.45,8.13,0.53,4.36,6.65,11.54,18.76,689.95,27.66,2.30,14.97,17.99,8.68,3.32,52.81,57.26,39.76,40.06,0.77,9.29,5.85,41.15,6.30,19.21,98.54,85.35,27.15,96.72,98.58,11.40,5.62,29.58,44.66,2.79,26.59,12.27,37.81,0.00,9.10,18.07,9.67,0.16,1.40,1.42,45543,44810,13828,90353,76525,89766,0.702,0.600,0.806,0.716,107863.0,4904.36943
2,39.00000,0.000000,70.39,2.15,24.8,82.57,6.87,10.09,0.62,16.78,66.22,15.59,58.34,9.86,1.05,20.42,4.93,0.51,7.27,8.51,21.20,31.01,457.17,32.98,0.67,45.49,6.46,3.62,0.00,39.38,44.10,30.24,36.91,0.52,6.25,3.60,24.03,7.79,22.12,95.49,91.63,19.93,99.14,96.36,3.31,0.84,41.25,33.08,1.97,38.52,21.53,54.79,1.12,5.47,7.03,16.59,0.65,3.10,3.64,3266,3047,3620,6313,2693,6303,0.650,0.559,0.757,0.650,43207.0,1813.00000
3,39.83177,0.401504,74.27,1.90,14.3,88.56,5.57,9.74,1.01,10.95,66.69,15.57,51.33,17.45,0.44,28.06,8.71,0.57,5.97,8.77,13.08,20.03,738.06,24.97,2.09,19.69,19.00,7.16,0.25,55.07,55.78,37.67,39.07,0.85,10.29,6.22,40.10,7.70,18.21,97.96,93.44,20.50,98.17,98.89,1.85,3.39,25.72,37.59,2.39,26.16,11.58,39.87,0.68,6.03,12.80,9.64,0.43,1.84,1.11,39124,39450,16653,78574,61921,78141,0.718,0.620,0.821,0.727,85359.0,2653.49875
4,36.75795,0.384490,72.94,2.12,18.1,86.72,6.44,10.22,1.14,13.51,72.15,12.93,49.74,10.77,2.02,24.06,7.09,0.50,4.72,6.93,13.70,22.08,577.18,28.46,2.36,26.67,14.08,8.60,0.06,45.54,49.50,34.10,38.39,0.57,8.53,7.22,24.74,9.77,25.05,97.53,86.87,16.66,91.89,98.85,5.34,0.85,35.81,38.73,1.46,32.21,15.53,46.96,0.00,1.77,25.88,10.16,0.38,1.43,1.15,8551,8478,2610,17029,14419,16942,0.692,0.602,0.799,0.688,16323.0,1519.32855


**Lembrete para testes de exportação de CSV**

In [ ]:
#idhmcovid_classif.to_csv (r'C:\Users\Jorge\Documents\TCC\idhm-pb-master\idhmcovid_class.csv', index = False, header=True)
#idhmcovid.to_csv (r'C:\Users\Jorge\Documents\TCC\idhm-pb-master\idhmcovid.csv', index = False, header=True)

**Separando previsores e classe**

In [153]:
previsores = idhmcovid_classif.iloc[:, 1:70]
classe_incid = idhmcovid_classif.iloc[:, 70].values
classe_mort = idhmcovid_classif.iloc[:, 0].values

**Transformação de atributos categóricos para numéricos**





In [ ]:
#from sklearn.preprocessing import LabelEncoder
#labelencoder = LabelEncoder()
#previsores.apply(LabelEncoder().fit_transform)

**Tratamento do dataframe de classe**

In [154]:
dataFrame = pd.DataFrame(data=classe_incid);
dataFrame.rename(columns = {0: 'niveis'}, inplace=True)
dataFrame.head()
categorias_incid = dataFrame.niveis

In [155]:
dataFrame2 = pd.DataFrame(data=classe_mort);
dataFrame2.rename(columns = {0: 'niveis'}, inplace=True)
dataFrame2.head()
categorias_mort = dataFrame2.niveis

In [156]:
categorias_incid = categorias_incid.values.reshape(-1, 1)
categorias_mort = categorias_mort.values.reshape(-1, 1)

**Visualização de quantos registros existem por classe**

In [157]:
unicos, quantidade = np.unique(categorias_incid, return_counts = True)
unicos

array([1.08436300e+01, 1.18497500e+01, 1.54966700e+01, ...,
       1.61887477e+04, 1.66403690e+04, 1.68099931e+04])

In [158]:
unicos2, quantidade2 = np.unique(categorias_mort, return_counts = True)
unicos2

array([  0.     ,   1.91421,   2.00698, ..., 229.73982, 233.00325,
       249.58403])

**Agrupamento com k-means, utilizando 3 clusters**

In [159]:
cluster_inc = KMeans(n_clusters = 3)
cluster_inc.fit(categorias_incid)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [160]:
cluster_mort = KMeans(n_clusters = 3)
cluster_mort.fit(categorias_mort)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

**Visualização dos três centroides**

In [166]:
centroides_inc = cluster_inc.cluster_centers_
centroides_inc

array([[2182.34548334],
       [5421.53425289],
       [ 798.43063429]])

In [167]:
centroides_mort = cluster_mort.cluster_centers_
centroides_mort

array([[11.86337725],
       [97.12017236],
       [44.26349611]])

**Visualização dos grupos que cada registro foi associado** (Agrupamento)

In [168]:
previsoes = cluster_inc.labels_
previsoes

array([0, 1, 0, ..., 2, 0, 1], dtype=int32)

In [169]:
previsoes2 = cluster_mort.labels_
previsoes2

array([2, 1, 2, ..., 0, 2, 1], dtype=int32)

In [170]:
dataFrame = pd.DataFrame(data=previsoes);
dataFrame.rename(columns = {0: 'niveis'}, inplace=True)
dataFrame.head()
categorias_incid = dataFrame.niveis

In [171]:
dataFrame2 = pd.DataFrame(data=previsoes2);
dataFrame2.rename(columns = {0: 'niveis'}, inplace=True)
dataFrame2.head()
categorias_mort = dataFrame2.niveis

**Contagem dos registros por classe**

In [172]:
unicos_inc, quantidade_inc = np.unique(previsoes, return_counts = True)
unicos_inc

array([0, 1, 2], dtype=int32)

In [173]:
quantidade_inc

array([2634,  431, 2500])

In [174]:
categorias_incid

0       0
1       1
2       0
3       0
4       0
       ..
5560    2
5561    0
5562    2
5563    0
5564    1
Name: niveis, Length: 5565, dtype: int32

In [175]:
unicos_mort, quantidade_mort = np.unique(previsoes, return_counts = True)
unicos_mort

array([0, 1, 2], dtype=int32)

In [176]:
quantidade_mort

array([2634,  431, 2500])

In [177]:
categorias_mort

0       2
1       1
2       2
3       2
4       2
       ..
5560    0
5561    2
5562    0
5563    2
5564    1
Name: niveis, Length: 5565, dtype: int32

**Exportar csv de previsores e classe para os classificadores**

In [ ]:
previsores.to_csv (r'/content/drive/My Drive/Colab Notebooks/covid-idhm/previsores.csv', index = False, header=True)
categorias_incid.to_csv (r'/content/drive/My Drive/Colab Notebooks/covid-idhm/classe_inc.csv', index = False, header=True)
categorias_mort.to_csv (r'/content/drive/My Drive/Colab Notebooks/covid-idhm/classe_mort.csv', index = False, header=True)
